# OpEn Rust Examples: Various Obstacles - Part 2

In [the previous part](https://github.com/inmo-jang/optimisation_tutorial/blob/master/tools_examples/OpEn/examples_rust/OpEn_Rust_examples_nonlinear_obstacles.ipynb), we had gone through about various shaped obstacles. As the next part, in this example, we are going to integrate the new obstacles with the path planner that we developed in [Example 09](https://github.com/inmo-jang/optimisation_tutorial/blob/master/tools_examples/OpEn/examples_rust/OpEn_Rust_examples_multiple_obstacles.ipynb).   

## Overall Architecture

We are going to build an unified generic path planner for various obstacles. To this end, I suggest an architecture that consists of:

- (1) **Obstacle functions**: $h$ functions that define the shape of any necessary obstacles. 

- (2) **`ProblemMaster` class**: a class that implements the optimisation problem we are going to address. This class should call the $h$ functions defined in (1). 

- (3) **Optimisation loop for each time step**

- (4) **The main function**




## 1) Obstacle Functions

Here, let's just remind of the mathematical formulations for some obstalces. For details, please refer to [here](https://github.com/inmo-jang/optimisation_tutorial/blob/master/tools_examples/OpEn/examples_rust/OpEn_Rust_examples_nonlinear_obstacles.ipynb). 



### Elipsoid

$$h(x) = 1-(x-c)^{\top}E(x-c)$$

where 
- $c \in \mathbb{R}^{n_d}$ is the centre of the elipsoid
- $E \in \mathbb{R}^{n_d \times n_d}$ is a diagonal matrix that controls the size of the elipsoid (i.e. $E = diag(\frac{1}{r_1^2},..., \frac{1}{r^2_{n_d}}$) )

In [2]:
// x: user position
// elip : (centre_x, centre_y, radius_x, radius_y)
fn h_elip(x: &[f64], elip: (f64, f64, f64, f64)) -> f64{
    let h = (1.0 - ((x[0]-elip.0)/elip.2).powi(2) - ((x[1]-elip.1)/elip.3).powi(2) ).max(0.0);  
    return h;
}

### Non-linear Obstacles

The followings are non-linear obtacles used in [Sathya et al., 2019](https://arxiv.org/pdf/1904.10546.pdf) (See Fig 3 in the paper).

The constraint for $O_1$ becomes

$$\prod_{i=1}^{m} [h^i(x)]^2_{+} = 0$$

where

$$h^1 := (y - c_y) - (x - c_x)^2 > 0$$
$$h^2 := 1 + \frac{(x - c_x)^2}{2} - (y - c_y) > 0$$

In [3]:
fn h_nlr_1(x: &[f64], centre: (f64, f64)) -> f64{
    // x: user position
    // centre : (c_x, c_y)        
    let h1 = ((x[1] - centre.1) - ((x[0] - centre.0)).powi(2)).max(0.0);
    let h2 = (1.0 + (x[0] - centre.0).powi(2)/2.0 - (x[1] - centre.1)).max(0.0);
    
    let h = h1*h2;
     
    return h;
}

The constraint for $O_2$ becomes

$$\prod_{i=1}^{m} [h^i(x)]^2_{+} = 0$$

where 

$$h_1 := (y - c_y) - 2 \sin(-\frac{x - c_x}{2})$$
$$h_{2-1} := 3\sin(\frac{x-c_x}{2} - 1) - (y-c_y)$$
$$h_{2-2} := x - c_x - 1$$
$$h_{2-3} := 8 - (x - c_x)$$

In [4]:
fn h_nlr_2(x: &[f64], centre: (f64, f64)) -> f64{
    // x: user position
    // centre : (c_x, c_y)    
    
    let h1 = ((x[1] - centre.1) - 2.0*(-(x[0] - centre.0)/2.0).sin()).max(0.0);
    let h2_1 = (3.0*(((x[0] - centre.0)/2.0) - 1.0).sin() - (x[1] - centre.1)).max(0.0);
    let h2_2 = (x[0] - centre.0 - 1.0).max(0.0);
    let h2_3 = (8.0 - (x[0] - centre.0)).max(0.0);
    
    let h = h1*h2_1*h2_2*h2_3;
       
    return h;
}

## 2) Problem Master

`ProblemMaster` will take arguments such as (a) **Robot start position**; (2) **Robot goal position**; (3) **Obstacle information**; and (4) **Robot dynamics**. 

Particularly for (3), note that the necessary information for different obstacles may be different. In this example, we just simply put `obstacle_info: Vec<(u32, f64, f64, f64, f64)>`, where the first element indicates **the obstacle type**; the others convey the obstacle shape information. As the nonlinear shaped obstacles defined above just need their central positions, the last two elements will be ignored for these obstacle types. 




#### OpEn Implementation

First, we need to import "optimization_engine". Also, we use "nalgebra" for linear algebra calculation. 
- In your local PC, it should be also declared in "Cargo.toml".
- Instead, in this jupyter notebook, we need to have "extern crate" as follows. 


In [5]:
extern crate optimization_engine;
use optimization_engine::{
    alm::*,
    constraints::*, panoc::*, *
};
extern crate nalgebra;
use nalgebra::base::{*};
// use nalgebra::base::{Matrix4, Matrix4x2, Matrix4x1};
// use std::cmp;

In [6]:
pub struct ProblemMaster{
    x_now: Matrix2x1<f64>, // Robot Start Position
    x_ref: Matrix2x1<f64>, // Robot Goal Position
    x_obs: Vec<(u32, f64, f64, f64, f64)>, // Obstacle Type and Shape Info
    u_max: f64, // Dyanmics Radius
}

impl ProblemMaster{
    pub fn init(_x_start: Matrix2x1<f64>, _x_ref: Matrix2x1<f64>, _x_obs: Vec<(u32, f64, f64, f64, f64)>, _u_max: f64) -> Self {
        let x_now = _x_start;
        let x_ref = _x_ref;
        let x_obs = _x_obs;
        let u_max = _u_max;
        Self{x_now, x_ref, x_obs, u_max}            
    }
    
    
    // ========= Cost function (You need to modify this) =========
    pub fn f_call(&self, u: &[f64]) -> f64{        
        let cost = (u[0]-self.x_ref[(0,0)]).powi(2) + (u[1]-self.x_ref[(1,0)]).powi(2);
        cost
    }
    // ===========================================================
    
    pub fn f(&self, u: &[f64], cost: &mut f64){
        *cost = self.f_call(u);        
    }
    
    // Gradient of the cost function
    pub fn df(&self, u: &[f64], grad: &mut [f64]){
        let f_0 = self.f_call(u);
        
        for i in 0..u.len() {
            let mut u_h = u.to_vec();
            u_h[i] += 0.000001;
            let f_h = self.f_call(u_h.as_slice());
            grad[i] = (-f_0 + f_h) / 0.000001;
        }
       
    } 
    
    // ========= F1 Constraint (You need to modify this) =========
    pub fn f1_call(&self, u: &[f64])-> Vec<f64> {
        let mut f1u = vec![0.0; u.len()];
        // Obstacle Avoidance Constraint (C2)
        let mut f1u_0: f64 = 0.0;        
        for j in 0..self.x_obs.len(){
            match self.x_obs[j].0 {
                // Elipsoid
                1 => f1u_0 += h_elip(u, (self.x_obs[j].1, self.x_obs[j].2, self.x_obs[j].3, self.x_obs[j].4)),
                // Non-linear shaped 1
                3 => f1u_0 += h_nlr_1(u, (self.x_obs[j].1, self.x_obs[j].2)),
                // Non-linear shaped 1
                4 => f1u_0 += h_nlr_2(u, (self.x_obs[j].1, self.x_obs[j].2)),
                _ => f1u_0 = 0.0,                
            };
        }
        f1u[0] = f1u_0;

        // Dynamics Constraint (C1)
        let mut u_now = Matrix2x1::new(0.0, 0.0);
        for i in 0..u.len(){
            u_now[(i,0)] = u[i];
        }
        let delta = u_now - self.x_now;
        f1u[1] = (delta.norm() - self.u_max).max(0.0);
        
        return f1u;
    }
    // ===========================================================
    
    pub fn f1(&self, u: &[f64], f1u: &mut [f64]){
        let f1u_vec = self.f1_call(u); 
        for i in 0..f1u_vec.len(){
            f1u[i] = f1u_vec[i];
        }
    }    
    
    // Jacobian of F1
    pub fn jf1_call(&self, u: &[f64])-> Matrix2<f64> {
        let mut jf1 = Matrix2::new(0.0, 0.0,
                              0.0, 0.0);
        
        let f1_0 = self.f1_call(u);

        for i in 0..f1_0.len(){
            for j in 0..u.len() {
                let mut u_h = u.to_vec();
                u_h[j] += 0.000001;
                let f_h = self.f1_call(u_h.as_slice());
                jf1[(i,j)] = (-f1_0[i] + f_h[i]) / 0.000001;
            }                        
        }

        return jf1;        
    } 
    
    // Jacobian Product (JF_1^{\top}*d)
    pub fn f1_jacobian_product(&self, u: &[f64], d: &[f64], res: &mut [f64]){
        let test = self.f1_call(u);
        
        let mut jf1_matrix = self.jf1_call(u);
        if test[0] < 0.0{ // Outside the obstacle
            jf1_matrix[(0,0)] = 0.0;
            jf1_matrix[(0,1)] = 0.0;  
        }          
        
        let mut d_matrix = Matrix2x1::new(0.0, 0.0);
        for i in 0..d.len(){
            d_matrix[(i,0)] = d[i];
        }
        
        let res_matrix =  jf1_matrix.transpose()*d_matrix;
        
        res[0] = res_matrix[(0,0)];
        res[1] = res_matrix[(1,0)];  
    }
    
}

## 3) Optimisation function for each time step


In [7]:
fn main_loop(_x_now: &[f64], _x_ref: &[f64], _x_obs: Vec<(u32, f64, f64, f64, f64)>, _u_max: f64) -> Vec<f64> {
 
    // ===========================================
    let mut x_now = Matrix2x1::new(0.0, 0.0);
    for i in 0.._x_now.len(){
        x_now[(i,0)] = _x_now[i];
    }
    
    let mut x_ref = Matrix2x1::new(0.0, 0.0);
    for i in 0.._x_ref.len(){
        x_ref[(i,0)] = _x_ref[i];
    }
    
    let x_obs = _x_obs;
    
    let u_max = _u_max;        
    
    let pm = ProblemMaster::init(x_now, x_ref, x_obs, u_max);
    
    // ===========================================
    
    let tolerance = 1e-5;
    let nx = 2; // problem_size: dimension of the decision variables
    let n1 = 2; // range dimensions of mappings F1
    let n2 = 0; // range dimensions of mappings F2
    let lbfgs_mem = 5; // memory of the LBFGS buffer
    
    // PANOCCache: All the information needed at every step of the algorithm
    let panoc_cache = PANOCCache::new(nx, tolerance, lbfgs_mem);
    
    // AlmCache: A cache structure that contains all the data 
    // that make up the state of the ALM/PM algorithm
    // (i.e., all those data that the algorithm updates)
    let mut alm_cache = AlmCache::new(panoc_cache, n1, n2);

    let set_c = Zero::new(); // Set C
    let bounds = Ball2::new(None, 1e12); // Set U
    let set_y = Ball2::new(None, 1e12);  // Set Y

    // ============= 
    // Re-define the functions linked to user parameters
    let f = |u: &[f64], cost: &mut f64| -> Result<(), SolverError> {
        pm.f(u, cost);
        Ok(())
    };
    
    let df = |u: &[f64], grad: &mut [f64]| -> Result<(), SolverError> {
        pm.df(u, grad);
        Ok(())
    };
    
    let f1 = |u: &[f64], f1u: &mut [f64]| -> Result<(), SolverError> {
        pm.f1(u, f1u);
        Ok(())
    };    
    
    let f1_jacobian_product = |u: &[f64], d: &[f64], res: &mut [f64]| -> Result<(), SolverError> {
        pm.f1_jacobian_product(u,d,res);
        Ok(())
    };      
    // ==============
    
    // AlmFactory: Prepare function psi and its gradient 
    // given the problem data such as f, del_f and 
    // optionally F_1, JF_1, C, F_2
    let factory = AlmFactory::new(
        f, // Cost function
        df, // Cost Gradient
        Some(f1), // MappingF1
        Some(f1_jacobian_product), // Jacobian Mapping F1 Trans
        NO_MAPPING, // MappingF2
        NO_JACOBIAN_MAPPING, // Jacobian Mapping F2 Trans
        Some(set_c), // Constraint set
        n2,
    );

    // Define an optimisation problem 
    // to be solved with AlmOptimizer
    let alm_problem = AlmProblem::new(
        bounds,
        Some(set_c),
        Some(set_y),
        |u: &[f64], xi: &[f64], cost: &mut f64| -> Result<(), SolverError> {
            factory.psi(u, xi, cost)
        },
        |u: &[f64], xi: &[f64], grad: &mut [f64]| -> Result<(), SolverError> {
            factory.d_psi(u, xi, grad)
        },
        Some(f1),
        NO_MAPPING,
        n1,
        n2,
    );

    let mut alm_optimizer = AlmOptimizer::new(&mut alm_cache, alm_problem)
        .with_delta_tolerance(1e-5)
        .with_max_outer_iterations(200)
        .with_epsilon_tolerance(1e-6)
        .with_initial_inner_tolerance(1e-2)
        .with_inner_tolerance_update_factor(0.5)
        .with_initial_penalty(100.0)
        .with_penalty_update_factor(1.05)
        .with_sufficient_decrease_coefficient(0.2)
        .with_initial_lagrange_multipliers(&vec![5.0; n1]);

    let mut u = _x_now.to_vec(); // Initial guess
    let solver_result = alm_optimizer.solve(&mut u);
    let r = solver_result.unwrap();
//     println!("\n\nSolver result : {:#.7?}\n", r);
//     println!("Solution u = {:#.6?}", u);
    
    return u;
    
}

## (4) Main Function Generating a trajectory 



In [8]:
fn main_instance(_x_now: &[f64], _x_ref: &[f64], _x_obs: Vec<(u32, f64, f64, f64, f64)>, _u_max: f64) -> Vec<(f64, f64)>  {
    
    let mut done = false;
    let mut x_now = _x_now.to_vec();
        
    let mut path_result = Vec::new();
    path_result.push((x_now[0], x_now[1]));
    println!("Solution x_now = {:#.6?}   {:#.6?}", x_now[0], x_now[1]);

    while !done{    
        let x_next = main_loop(x_now.as_slice(), _x_ref, _x_obs.clone(), _u_max);

        x_now = x_next;
        let delta = ((x_now[0]-_x_ref[0]).powi(2)+(x_now[1]-_x_ref[1]).powi(2)).sqrt();
        if delta < 0.00001{
            done = true;
        }
        path_result.push((x_now[0], x_now[1]));
        println!("Solution x_now = {:#.6?}   {:#.6?}", x_now[0], x_now[1]);   
    }
          
    return path_result;
            
        
}

### Result

#### Case (1)
- $x_{start} = [0.0, .0]$;  
- $x_{goal} = [30.0, 30.0]$; 
- $u_{max} = 0.1$. 

In [11]:
let pos_start = &[0.0_f64, 0.0_f64];
let pos_goal = &[30.0_f64, 30.0_f64];
let mut pos_obstacle = Vec::new();
pos_obstacle.push((1_u32, 3.0_f64, 4.0_f64, 1.5_f64, 2.0_f64)); 
pos_obstacle.push((1_u32, 23.0_f64, 23.0_f64, 2.5_f64, 1.0_f64)); 
pos_obstacle.push((3_u32, 29.0_f64, 27.0_f64, 0.0_f64, 0.0_f64));
pos_obstacle.push((4_u32, 9.0_f64, 12.0_f64, 0.0_f64, 0.0_f64));
let max_movement = 0.1_f64;

In [12]:
let path = main_instance(pos_start, pos_goal, pos_obstacle.clone(), max_movement);

Solution x_now = 0.000000   0.000000
Solution x_now = 0.070711   0.070711
Solution x_now = 0.141421   0.141421
Solution x_now = 0.212136   0.212136
Solution x_now = 0.282846   0.282846
Solution x_now = 0.353556   0.353556
Solution x_now = 0.424266   0.424266
Solution x_now = 0.494977   0.494977
Solution x_now = 0.565688   0.565688
Solution x_now = 0.636398   0.636398
Solution x_now = 0.707109   0.707109
Solution x_now = 0.777827   0.777827
Solution x_now = 0.848537   0.848537
Solution x_now = 0.919247   0.919247
Solution x_now = 0.989959   0.989959
Solution x_now = 1.060669   1.060669
Solution x_now = 1.131382   1.131382
Solution x_now = 1.202096   1.202096
Solution x_now = 1.272810   1.272810
Solution x_now = 1.343522   1.343522
Solution x_now = 1.414235   1.414235
Solution x_now = 1.484945   1.484945
Solution x_now = 1.555655   1.555655
Solution x_now = 1.626365   1.626365
Solution x_now = 1.697075   1.697075
Solution x_now = 1.767785   1.767785
Solution x_now = 1.838495   1.838495
S

Solution x_now = 12.273784   10.004365
Solution x_now = 12.274250   10.004396
Solution x_now = 12.274612   10.004420
Solution x_now = 12.275000   10.004445
Solution x_now = 12.275372   10.004470
Solution x_now = 12.275761   10.004496
Solution x_now = 12.276114   10.004520
Solution x_now = 12.276451   10.004543
Solution x_now = 12.276788   10.004565
Solution x_now = 12.277178   10.004592
Solution x_now = 12.277575   10.004619
Solution x_now = 12.277920   10.004642
Solution x_now = 12.278295   10.004668
Solution x_now = 12.278722   10.004697
Solution x_now = 12.279066   10.004721
Solution x_now = 12.279436   10.004746
Solution x_now = 12.279797   10.004771
Solution x_now = 12.280147   10.004795
Solution x_now = 12.280509   10.004820
Solution x_now = 12.280850   10.004844
Solution x_now = 12.281180   10.004867
Solution x_now = 12.281516   10.004890
Solution x_now = 12.281877   10.004916
Solution x_now = 12.282260   10.004943
Solution x_now = 12.282628   10.004968
Solution x_now = 12.28299

Solution x_now = 12.909329   10.145552
Solution x_now = 12.952996   10.162348
Solution x_now = 12.997091   10.180195
Solution x_now = 12.997381   10.180316
Solution x_now = 13.019549   10.189626
Solution x_now = 13.041587   10.199100
Solution x_now = 13.041873   10.199225
Solution x_now = 13.042178   10.199359
Solution x_now = 13.042452   10.199478
Solution x_now = 13.042743   10.199605
Solution x_now = 13.043026   10.199728
Solution x_now = 13.062063   10.208102
Solution x_now = 13.084427   10.218148
Solution x_now = 13.172169   10.259697
Solution x_now = 13.203796   10.275499
Solution x_now = 13.270394   10.310179
Solution x_now = 13.270658   10.310320
Solution x_now = 13.270945   10.310475
Solution x_now = 13.291229   10.321415
Solution x_now = 13.378329   10.370347
Solution x_now = 13.464335   10.421366
Solution x_now = 13.548767   10.474944
Solution x_now = 13.632119   10.530178
Solution x_now = 13.714307   10.587143
Solution x_now = 13.795379   10.645666
Solution x_now = 13.87541

Solution x_now = 26.299123   24.141996
Solution x_now = 26.352559   24.226531
Solution x_now = 26.405995   24.311067
Solution x_now = 26.459430   24.395601
Solution x_now = 26.512866   24.480136
Solution x_now = 26.566303   24.564672
Solution x_now = 26.619746   24.649202
Solution x_now = 26.673182   24.733736
Solution x_now = 26.726617   24.818270
Solution x_now = 26.780052   24.902804
Solution x_now = 26.833487   24.987339
Solution x_now = 26.886923   25.071872
Solution x_now = 26.940354   25.156405
Solution x_now = 26.993784   25.240938
Solution x_now = 27.047214   25.325471
Solution x_now = 27.100647   25.410006
Solution x_now = 27.154067   25.494545
Solution x_now = 27.207497   25.579078
Solution x_now = 27.260937   25.663609
Solution x_now = 27.314369   25.748145
Solution x_now = 27.367801   25.832681
Solution x_now = 27.421234   25.917216
Solution x_now = 27.474665   26.001752
Solution x_now = 27.528096   26.086288
Solution x_now = 27.581529   26.170819
Solution x_now = 27.63496

The result can be visualised as follows. 

![](https://raw.githubusercontent.com/inmo-jang/optimisation_tutorial/master/tools_examples/OpEn/examples_rust/example_11_nonlinear_obstacles/path_result.svg)



- **NOTE**: [The source file of this example](https://github.com/inmo-jang/optimisation_tutorial/tree/master/tools_examples/OpEn/examples_rust/example_11_nonlinear_obstacles) is slightly different than here. Using the source file, you can plot the graphs above. 





## Future work

- Let's make the `ProblemMaster` have resizable parameters (e.g. not binding to `Matrix2x1` with consideration of 2D and 3D). Maybe it is simpler to use **tuple**. 

- Let's consider a longer time horizon. 

- Let's consider a system dynamics. 


